# Deep learing

## FNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

In [ ]:
cols = ["G" , "C" , "B" , "A" , "Ia" , "Ib" , "Ic" , "Va" , "Vb" , "Vc"] 
# Ia" , "Ib" , "Ic" , "Va" , "Vb" , "Vc is Input : features
# G , C , B , A is OutPut : labels
df = pd.read_csv("../../classData.csv")
cols[4:]
# Step 1: Encode fault combinations into single-class labels
fault_map = {
    '0000': 0,  # No Fault
    '1001': 1,  # LG
    '0011': 2,  # LL
    '0110': 3 , # Unkown : 3
    '1011': 4,  # LLG
    '0111': 5,  # LLL
    '1111': 6   # LLLG
}
df

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc
0,1,0,0,1,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815
1,1,0,0,1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099
2,1,0,0,1,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428
3,1,0,0,1,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570
4,1,0,0,1,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983
...,...,...,...,...,...,...,...,...,...,...
7856,0,0,0,0,-66.237921,38.457041,24.912239,0.094421,-0.552019,0.457598
7857,0,0,0,0,-65.849493,37.465454,25.515675,0.103778,-0.555186,0.451407
7858,0,0,0,0,-65.446698,36.472055,26.106554,0.113107,-0.558211,0.445104
7859,0,0,0,0,-65.029633,35.477088,26.684731,0.122404,-0.561094,0.438690


In [41]:
df['fault_type'] = df[['G', 'C', 'B', 'A']].astype(str).agg(''.join, axis=1)
# Map to single class label
df['fault_class'] = df['fault_type'].map(fault_map)
# Drop rows with unknown fault combinations


In [42]:
# Keep dropped rows (unknown fault types) in a separate DataFrame
df_unknown = df[~df['fault_type'].isin(fault_map.keys())].copy()

# Filter valid rows for training
df = df[df['fault_type'].isin(fault_map.keys())].copy()


In [43]:
df 

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,fault_type,fault_class
0,1,0,0,1,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815,1001,1
1,1,0,0,1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099,1001,1
2,1,0,0,1,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428,1001,1
3,1,0,0,1,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570,1001,1
4,1,0,0,1,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983,1001,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7856,0,0,0,0,-66.237921,38.457041,24.912239,0.094421,-0.552019,0.457598,0000,0
7857,0,0,0,0,-65.849493,37.465454,25.515675,0.103778,-0.555186,0.451407,0000,0
7858,0,0,0,0,-65.446698,36.472055,26.106554,0.113107,-0.558211,0.445104,0000,0
7859,0,0,0,0,-65.029633,35.477088,26.684731,0.122404,-0.561094,0.438690,0000,0


In [44]:
df_unknown

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,fault_type,fault_class


In [52]:
train , valid , test = np.split(df.sample(frac = 1) , [int(0.6 * len(df)) , int(0.8 * len(df))])
# train  60% |||||||||||||
# valid  20%  1234567890
# test 20%
test

c:\Users\OussamaTab\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,fault_type,fault_class
6355,0,0,0,0,14.501390,-27.946364,10.131196,0.584086,-0.194328,-0.389758,0000,0
7473,0,0,0,0,-58.054047,22.698247,32.379605,0.247586,-0.585507,0.337921,0000,0
436,1,0,0,1,-877.795362,-52.814260,-1.321478,0.062310,0.204516,-0.266825,1001,1
5255,1,1,1,1,-571.429576,-298.122042,869.549355,-0.041354,0.028742,0.012612,1111,6
6341,0,0,0,0,1.059004,-24.355277,19.978034,0.590857,-0.312708,-0.278148,0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1551,1,0,1,1,-847.837440,617.905170,-0.119805,-0.010499,0.232221,-0.221722,1011,4
4525,1,1,1,1,349.280107,-888.845717,539.594306,-0.026187,-0.015348,0.041535,1111,6
5088,1,1,1,1,162.531290,670.665657,-833.199211,0.040467,-0.009449,-0.031017,1111,6
3639,0,1,1,1,-463.528416,883.641844,-417.999189,0.021580,0.020783,-0.042363,0111,5


In [60]:
def scale_dataset(dataFrame):
    from tensorflow.keras.utils import to_categorical
    x = dataFrame[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']].values
    # y = dataFrame[['G', 'C', 'B', 'A']].values
    y = df[['fault_class']].astype(int).values

    scaler = StandardScaler() 
    x = scaler.fit_transform(x)

    # data = np.hstack((x, y))
    # Step 4: One-hot encode the target for categorical classification
    y_cat = to_categorical(y, num_classes=7)

    return  x, y

In [61]:
X_train , Y_train = scale_dataset(train)
X_valid , Y_valid = scale_dataset(valid)
X_test , Y_test = scale_dataset(test)

test

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,fault_type,fault_class
6355,0,0,0,0,14.501390,-27.946364,10.131196,0.584086,-0.194328,-0.389758,0000,0
7473,0,0,0,0,-58.054047,22.698247,32.379605,0.247586,-0.585507,0.337921,0000,0
436,1,0,0,1,-877.795362,-52.814260,-1.321478,0.062310,0.204516,-0.266825,1001,1
5255,1,1,1,1,-571.429576,-298.122042,869.549355,-0.041354,0.028742,0.012612,1111,6
6341,0,0,0,0,1.059004,-24.355277,19.978034,0.590857,-0.312708,-0.278148,0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1551,1,0,1,1,-847.837440,617.905170,-0.119805,-0.010499,0.232221,-0.221722,1011,4
4525,1,1,1,1,349.280107,-888.845717,539.594306,-0.026187,-0.015348,0.041535,1111,6
5088,1,1,1,1,162.531290,670.665657,-833.199211,0.040467,-0.009449,-0.031017,1111,6
3639,0,1,1,1,-463.528416,883.641844,-417.999189,0.021580,0.020783,-0.042363,0111,5


In [62]:
# Step 6: Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(6, activation='softmax')  # 6 classes for multiclass classification
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\OussamaTab\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
# Step 7: Train the model
model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 6)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert softmax output to predicted class
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(Y_test, y_pred_classes))
print(confusion_matrix(Y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     478.0
           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       0.0
           6       0.00      0.00      0.00     209.0
           7       0.00      0.00      0.00     216.0
           8       0.00      0.00      0.00       0.0
           9       0.00      0.00      0.00     228.0
          10       0.00      0.00      0.00       0.0
          11       0.00      0.00      0.00     219.0
          12       0.00      0.00      0.00       0.0
          13       0.00      0.00      0.00       0.0
          14       0.00      0.00      0.00       0.0
          15       0.00      0.00      0.00     223.0

    accuracy                           0.00    1573.0
   macro avg       0.00      0.00      0.00    1573.0
weighted avg       0.00      0.00      0.00    1573.0

[[  0   0   0   6   0   0

c:\Users\OussamaTab\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\OussamaTab\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\OussamaTab\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver